In [1]:
import numpy as np  
import networkx as nx
import scipy.sparse as sp
import scipy as sc 
import timeit
import time
import pandas as pd 
import tqdm
from planetoid_loader import load_data 
from sklearn import metrics  as m
from scipy.sparse.linalg import gmres 
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import  cosine_similarity , rbf_kernel

In [2]:
adj, attr_matrix, y_train, y_val, y_test, train_mask, val_mask, test_mask, ally, allx = load_data('citeseer')
Af = adj.toarray()
nnodes = Af.shape[0]

### Covariance betwen nodes 

In [89]:
Xn = np.copy(attr_matrix.toarray())
Xn = Xn.T  
Xn = Xn - np.median(Xn, axis=0)
MMx =  np.dot(Xn.T, Xn) / (Xn.shape[0] - 1)  

### Cos distance $W_{COS}$ $W_{RBF}$

In [5]:
MMCos = cosine_similarity(attr_matrix.toarray()) / (Xn.shape[0]-1) 
MMrbf = rbf_kernel(attr_matrix.toarray())/(Xn.shape[0] -1) 

In [68]:
def calc_A_hat(adj_matrix: sp.spmatrix, delta, sigma, MMx) -> sp.spmatrix:
    nnodes = adj_matrix.shape[0]
    A = adj_matrix + sp.eye(nnodes)#Ω#@ D_invsqrt_corr
    D_vec = np.sum(A, axis=1).A1 
    lsigma = sigma - 1
    rsigma = - sigma
    wsigma = -2*sigma + 1
    
    D_l = sp.diags(np.power(D_vec, lsigma)) 
    D_r= sp.diags(np.power(D_vec, rsigma ) )
    Dw = sp.diags(np.power(D_vec, wsigma ) )
    S_ = MMx@ Dw 
    
    return S_ , D_l@A@D_r  +   delta* S_

### Parameters for PRPCA

In [115]:
delta = 1
sigma = 1
alpha= 0.9
svd_error = 0.1
iter_ = 15
tol = 1e-03
beta = 0.9
svd = TruncatedSVD(n_components=1, algorithm='randomized')

mmc, AHAT = calc_A_hat(Af, delta, sigma, MMx ) 
rex = (np.identity(nnodes)  - alpha * AHAT )
svd.fit(mmc)
gamma = svd.singular_values_[0] 

### PRPCA

In [118]:
def JOR(A, Z, Y, iter_, beta, alpha):
    A = np.copy(A)
    Z = np.copy(Z)
    Y = np.copy(Y) 
    for _ in range(iter_):
        Z = Z + beta*(alpha * AHAT@Z - Z + (1-alpha) * Y) 
    return Z
def GMRES(A, Y, alpha, k, tol):
    A = np.copy(A)
    Y = np.copy(Y) 
    predicts = []
    for j in range(k): 
        temp_ = gmres(A, (1-alpha)*Y[:,j], tol=tol)[0] 
        predicts.append([temp_])
    return np.concatenate(predicts).T 

if (1 + delta * (gamma - svd_error)) <= 1/alpha: 
    print('jor')
    start = timeit.default_timer()
    Z = JOR(A=AHAT, Z=y_train, Y=y_train, iter_=iter_, beta=beta, alpha=alpha)
    print(timeit.default_timer() - start)
else:
    print('gmres')
    start = timeit.default_timer()
    Z = GMRES(A=rex, Y=y_train, alpha=alpha, k=y_train.shape[1], tol=tol)
    print(timeit.default_timer() - start)
    
m.f1_score(np.argmax(y_test[test_mask], axis=-1),
                  np.argmax(np.array(Z  )[test_mask], axis=-1), average='micro') 